In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style = 'darkgrid')

import geopandas as gpd
from keplergl import KeplerGl

_________
**Reading in Accident Data**
__________

In [3]:
# Read in Dataframe with All the accident data
acc_df = pd.read_csv('../data/US_Accidents_Dec20.csv')

In [4]:
acc_df.head(1)

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night


In [5]:
# lower and eliminate spaces in column names
acc_df.columns = acc_df.columns.str.lower().str.replace(' ', '_')

_________
**Read in License Data**, Format into DF, Prepare for Concatenation
________

In [6]:
# Read in Dataframe with data on driver's licensing in each state
license_df = pd.read_csv('../data/licensed_drivers_by_state.csv')

In [7]:
license_df.head(1)

,STATE,UNDER,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,OVER,total_drivers
0,Alabama,"219,648","327,163","347,679","320,583","307,331","292,967","311,675","309,970","339,239","327,915","286,986","242,004","168,934","113,483","110,574","4,026,151"


In [8]:
# lower and eliminate spaces in column names for license_df
license_df.columns = license_df.columns.str.lower().str.replace(' ', '_')

In [9]:
# creating df with only state column and total drivers columns
license_df = license_df.loc[:, ['state', 'total_drivers']]

In [10]:
license_df['state'] = license_df['state'].str.lower()

In [11]:
# replacing dist of col. with district of columbia so I can concatenate with other dfs
license_df['state'].replace('dist. of col. ', 'district of columbia', inplace = True)

In [12]:
# setting the state to index and dropping alaska and hawaii since they're not in accident data
license_df.set_index('state', inplace = True)
license_df.drop(['alaska', 'hawaii'], inplace = True)

In [13]:
license_df.shape

(49, 1)

In [14]:
# sorting index alphabetically and ensuring proper format
license_df.sort_index(inplace = True)
license_df.index = license_df.index.str.lower().str.strip()

______
**Read in Shape Files**, Format into DF and Prepare for Concatenation
______

In [15]:
national_shape = gpd.read_file('../data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp')

In [16]:
# Dropping island territories
national_shape.drop([13, 27, 37, 38, 42, 44, 45], inplace = True)

In [17]:
# lower and eliminate spaces in column names
national_shape.columns = national_shape.columns.str.lower().str.replace(' ', '_')

In [18]:
# converting state name column to lowercase
national_shape['name'] = national_shape['name'].str.lower()

In [19]:
national_shape.shape

(49, 10)

In [20]:
# Setting index to name of state to prepare for concatenation
national_shape.set_index(keys = 'name', inplace = True)

In [21]:
# Sorting index alphabetically and ensuring proper format
national_shape.sort_index(inplace = True)
national_shape.index = national_shape.index.str.lower().str.strip()

________
**Create State Accident Counts**, Form into DF and Prepare for Concatenation
________

In [22]:
# Grouping acc_df by state and getting count of all accidents by state
state_count = acc_df.groupby('state').count()
state_count.head()

,id,source,tmc,severity,start_time,end_time,start_lat,start_lng,end_lat,end_lng,...,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight
state,,,,,,,,,,,,,,,,,,,,,
AL,56989,56989,47642,56989,56989,56989,56989,56989,9347,9347,...,56989,56989,56989,56989,56989,56989,56989,56989,56989,56989
AR,5089,5089,732,5089,5089,5089,5089,5089,4357,4357,...,5089,5089,5089,5089,5089,5089,5089,5089,5089,5089
AZ,93038,93038,62918,93038,93038,93038,93038,93038,30120,30120,...,93038,93038,93038,93038,93038,93038,93038,93038,93038,93038
CA,971856,971856,523752,971856,971856,971856,971856,971856,448104,448104,...,971856,971856,971856,971856,971856,971856,971848,971848,971848,971848
CO,54028,54028,34226,54028,54028,54028,54028,54028,19802,19802,...,54028,54028,54028,54028,54028,54028,54025,54025,54025,54025


In [23]:
# resetting index to numeric so I can map full state names to abbreviated names
state_count = state_count.reset_index()

In [24]:
state_count.head()

,state,id,source,tmc,severity,start_time,end_time,start_lat,start_lng,end_lat,...,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight
0,AL,56989,56989,47642,56989,56989,56989,56989,56989,9347,...,56989,56989,56989,56989,56989,56989,56989,56989,56989,56989
1,AR,5089,5089,732,5089,5089,5089,5089,5089,4357,...,5089,5089,5089,5089,5089,5089,5089,5089,5089,5089
2,AZ,93038,93038,62918,93038,93038,93038,93038,93038,30120,...,93038,93038,93038,93038,93038,93038,93038,93038,93038,93038
3,CA,971856,971856,523752,971856,971856,971856,971856,971856,448104,...,971856,971856,971856,971856,971856,971856,971848,971848,971848,971848
4,CO,54028,54028,34226,54028,54028,54028,54028,54028,19802,...,54028,54028,54028,54028,54028,54028,54025,54025,54025,54025


In [25]:
state_remap = {'AL': 'alabama', 'AR': 'arkansas', 'AZ': 'arizona', 'CA': 'california', 'CO': 'colorado',
 'CT': 'connecticut', 'DC': 'district of columbia', 'DE': 'delaware', 'FL': 'florida',
 'GA': 'georgia', 'IA': 'iowa', 'ID': 'idaho', 'IL': 'illinois',
 'IN': 'indiana', 'KS': 'kansas', 'KY': 'kentucky', 'LA': 'louisiana',
 'MA': 'massachusetts', 'MD': 'maryland', 'ME': 'maine', 'MI': 'michigan',
 'MN': 'minnesota', 'MO': 'missouri', 'MS': 'mississippi', 'MT': 'montana',
 'NC': 'north carolina', 'ND': 'north dakota', 'NE': 'nebraska',
 'NH': 'new hampshire', 'NJ': 'new jersey', 'NM': 'new mexico', 'NV': 'nevada',
 'NY': 'new york', 'OH': 'ohio', 'OK': 'oklahoma', 'OR': 'oregon',
 'PA': 'pennsylvania', 'RI': 'rhode island', 'SC': 'south carolina',
 'SD': 'south dakota', 'TN': 'tennessee', 'TX': 'texas', 'UT': 'utah',
 'VA': 'virginia', 'VT': 'vermont', 'WA': 'washington',
 'WI': 'wisconsin', 'WV': 'west virginia', 'WY':  'wyoming'}

In [26]:
# mapping full names to abbreviated state names
state_count['state'] = state_count['state'].map(state_remap)

In [27]:
# renaming id column to count.
state_count.rename(columns = {'id': 'count', }, inplace = True)

In [28]:
# eliminating all columns except count and state
state_count = state_count.loc[ : , ['state', 'count']]

In [29]:
state_count.head()

,state,count
0,alabama,56989
1,arkansas,5089
2,arizona,93038
3,california,971856
4,colorado,54028


In [30]:
# resetting index to state so that it can be concatenated with license data
state_count.set_index(keys = 'state', inplace = True)

In [31]:
# sorting index alphabetically and ensuring proper format
state_count.sort_index(inplace = True)
state_count.index = state_count.index.str.lower().str.strip()

In [32]:
state_count.shape

(49, 1)

______
**Concatenating Grouped DF's**
_______

In [33]:
national_group = pd.concat([state_count, national_shape, license_df], axis = 1, verify_integrity = True)
national_group.shape

(49, 11)

In [34]:
national_group.columns

Index(['count', 'statefp', 'statens', 'affgeoid', 'geoid', 'stusps', 'lsad',
       'aland', 'awater', 'geometry', 'total_drivers'],
      dtype='object')

In [35]:
national_group.head()

,count,statefp,statens,affgeoid,geoid,stusps,lsad,aland,awater,geometry,total_drivers
alabama,56989,01,01779775,0400000US01,01,AL,00,131174048583,4593327154,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...","4,026,151"
arizona,93038,04,01779777,0400000US04,04,AZ,00,294198551143,1027337603,"POLYGON ((-114.81629 32.50804, -114.81432 32.5...","5,369,210"
arkansas,5089,05,00068085,0400000US05,05,AR,00,134768872727,2962859592,"POLYGON ((-94.61783 36.49941, -94.61765 36.499...","2,153,929"
california,971856,06,01779778,0400000US06,06,CA,00,403503931312,20463871877,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...","27,213,650"
colorado,54028,08,01779779,0400000US08,08,CO,00,268422891711,1181621593,"POLYGON ((-109.06025 38.59933, -109.05954 38.7...","4,235,384"


In [45]:
national_group['total_drivers'] = national_group['total_drivers'].str.replace(',', '')

In [48]:
national_group['total_drivers'] = national_group['total_drivers'].astype(int)

In [49]:
national_group['acc_per_cap'] = national_group['count'] / national_group['total_drivers']

In [50]:
national_group

,count,statefp,statens,affgeoid,geoid,stusps,lsad,aland,awater,geometry,total_drivers,acc_per_cap
alabama,56989,01,01779775,0400000US01,01,AL,00,131174048583,4593327154,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ...",4026151,0.014155
arizona,93038,04,01779777,0400000US04,04,AZ,00,294198551143,1027337603,"POLYGON ((-114.81629 32.50804, -114.81432 32.5...",5369210,0.017328
arkansas,5089,05,00068085,0400000US05,05,AR,00,134768872727,2962859592,"POLYGON ((-94.61783 36.49941, -94.61765 36.499...",2153929,0.002363
california,971856,06,01779778,0400000US06,06,CA,00,403503931312,20463871877,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987...",27213650,0.035712
colorado,54028,08,01779779,0400000US08,08,CO,00,268422891711,1181621593,"POLYGON ((-109.06025 38.59933, -109.05954 38.7...",4235384,0.012756
connecticut,32488,09,01779780,0400000US09,09,CT,00,12542497068,1815617571,"MULTIPOLYGON (((-72.76143 41.24233, -72.75973 ...",2608061,0.012457
delaware,6907,10,01779781,0400000US10,10,DE,00,5045925646,1399985648,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ...",812529,0.008501
district of columbia,6505,11,01702382,0400000US11,11,DC,00,158340391,18687198,"POLYGON ((-77.11976 38.93434, -77.11253 38.940...",535579,0.012146
florida,369559,12,00294478,0400000US12,12,FL,00,138949136250,31361101223,"MULTIPOLYGON (((-80.17628 25.52505, -80.17395 ...",15560628,0.023750
georgia,107071,13,01705317,0400000US13,13,GA,00,149482048342,4422936154,"MULTIPOLYGON (((-81.27939 31.30792, -81.27716 ...",7261266,0.014746


_________
**Plotting**
__________

# Kepler.gl Plot
Created a basic national map with accident count, licensed drivers and percentage of accidents per capita (derived by dividing accident count by licensed drivers per state)


In [54]:
ng_gdf = gpd.GeoDataFrame(national_group)

In [55]:
ng_gdf = ng_gdf.loc[:, ['count', 'geometry', 'total_drivers', 'acc_per_cap']]

In [56]:
# Instantiate KeplerGl
map_1 = KeplerGl(height = 400)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [57]:
# Add data to KeplerGl instance
map_1.add_data(data = ng_gdf, name = 'NationalAccidents')

**Plot Detailing Number of Accidents Per State, Total Licensed Drivers Per State and Accidents Per Capita**

In [58]:
display(map_1)

KeplerGl(data={'NationalAccidents': {'index': ['alabama', 'arizona', 'arkansas', 'california', 'colorado', 'co…

In [53]:
# Converting acc_df to geopandas df. 
# This also converts latitude and longitude columns to geomtry columns (these are shapely objects)
# https://gis.stackexchange.com/questions/174159/converting-pandas-dataframe-to-geodataframe/258376#258376
gdf = gpd.GeoDataFrame(acc_df, 
                             geometry=gpd.points_from_xy(x=acc_df['start_lng'], y=acc_df['start_lat']))

In [50]:
map_2 = KeplerGl(height = 400)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [51]:
# Add data to KeplerGl instance
map_2.add_data(data = gdf, name = 'Accidents')

In [52]:
display(map_2)

In [59]:
location_df = acc_df.loc[:, ['start_lat', 'start_lng']]

In [60]:
loc_gdf = gpd.GeoDataFrame(location_df, 
                             geometry=gpd.points_from_xy(x=acc_df['start_lng'], y=acc_df['start_lat']))

In [61]:
map_3 = KeplerGl(height = 400)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [62]:
map_3.add_data(data = gdf, name = "Accident Location")

KeyboardInterrupt: 

In [ ]:
display(map_3)

In [1]:
ng_gdf.plot(figsize = (20,20));

NameError: name 'ng_gdf' is not defined

In [ ]:
national_shape.plot(, column='id', cmap='Oranges', figsize=(40, 80))
acc_df.plot(figsize = (40, 80))
plt.show()

In [ ]:
us_acc.plot(kind = "scatter", x="Start_Lng",y="Start_Lat",alpha = 0.009)

In [ ]:
us_acc.plot(kind = "scatter", x="Start_Lng",y="Start_Lat",alpha = 0.009,c="Severity", 
            cmap=plt.get_cmap("jet"), colorbar = False, figsize=(15,7))
plt.figure(figsize=(20,12))
plt.show()